In [1]:
# =========================
# CÉLULA 0 — VETORIZAR JSON DE SCHEMA (OFFLINE)
# =========================
# Use esta célula só quando mudar algum arquivo em schema/*.json.
# Ela recria:
#   schema_vetorizado/schema_emb.npy
#   schema_vetorizado/schema_chunks.json

import json
import os
from pathlib import Path

import numpy as np
import ollama

# -------------------------
# ARQUIVOS DE ENTRADA (SCHEMAS)
# -------------------------
SCHEMA_DIR = Path("schema")

ENTITY_TO_FILE = {
    "Agravo": "Agravo.json",
    "AtividadeExec": "AtividadeExec.json",
    "AtividadeResumo": "AtividadeResumo.json",
    "Casos": "Casos.json",
    "Dia": "Dia.json",
    "Meteo": "Meteo.json",
    "Municipio": "Municipio.json",
    "Notificacao": "Notificacao.json",
    "TipoAtividade": "TipoAtividade.json",
}

# -------------------------
# ARQUIVOS DE SAÍDA (ÍNDICE VETORIAL)
# -------------------------
INDEX_DIR = Path("schema_vetorizado")
INDEX_DIR.mkdir(exist_ok=True)

EMB_PATH = INDEX_DIR / "schema_emb.npy"
CHUNKS_PATH = INDEX_DIR / "schema_chunks.json"

# Modelo de embedding
EMBED_MODEL_NAME = os.environ.get("EMBED_MODEL_NAME", "bge-m3")


def _truncate(text, max_len):
    if not text:
        return ""
    text = str(text).strip().replace("\n", " ")
    if len(text) <= max_len:
        return text
    return text[: max_len - 3] + "..."


def _load_schema_json(filename):
    path = SCHEMA_DIR / filename
    if not path.exists():
        print(f"[WARN] Schema não encontrado: {path}")
        return None
    try:
        with path.open(encoding="utf-8") as f:
            data = json.load(f)
        if not isinstance(data, dict):
            return None
        return data
    except Exception as e:
        print(f"[WARN] Não foi possível ler schema {filename}: {e}")
        return None


def _schema_to_chunks_for_embedding(schema, filename):
    """
    Converte um schema JSON em vários 'chunks' semânticos:
      - 1 chunk para a entidade
      - 1 chunk por campo
      - 1 chunk por relacionamento
      - CASO ESPECIAL AtividadeExec: 1 chunk por tipo (1..16)
        com campos_exclusivos e campos_comuns.
    """
    no = schema.get("no") or {}
    label = no.get("label") or filename.replace(".json", "")

    chunks = []

    # 1) Descrição da entidade
    descr_ent = (no.get("descricao") or schema.get("descricao_geral") or "").strip()
    descr_ent_short = _truncate(descr_ent, 256)
    if descr_ent_short:
        text = f"Entidade {label}. {descr_ent_short}"
        chunks.append({
            "entity": label,
            "filename": filename,
            "kind": "entity",
            "name": label,
            "descr": descr_ent_short,
            "text": text,
        })

    # 2) Campos
    props = {}

    # comuns
    if isinstance(no.get("propriedades"), dict):
        for name, meta in no["propriedades"].items():
            props[name] = meta or {}

    # declarados dentro de tipos (AtividadeExec)
    if isinstance(no.get("tipos"), dict):
        for _, tipo_info in no["tipos"].items():
            for name, meta in (tipo_info.get("propriedades") or {}).items():
                props.setdefault(name, meta or {})

    for name, meta in props.items():
        meta = meta or {}
        tipo = (meta.get("tipo") or "").strip()
        descr = (meta.get("descricao") or "").strip()
        descr_short = _truncate(descr, 120)

        text_parts = [f"Campo {name} da entidade {label}."]
        if tipo:
            text_parts.append(f"Tipo: {tipo}.")
        if descr_short:
            text_parts.append(descr_short)
        text = " ".join(text_parts)

        chunks.append({
            "entity": label,
            "filename": filename,
            "kind": "field",
            "name": name,
            "tipo": tipo,
            "descr": descr_short,
            "text": text,
        })

    # 3) Relacionamentos
    rels = schema.get("relacoes") or []
    for r in rels:
        tipo_rel = (r.get("tipo") or "").strip()
        de = (r.get("de") or "").strip()
        para = (r.get("para") or "").strip()
        if not (tipo_rel and de and para):
            continue

        descr = (r.get("descricao") or "").strip()
        descr_short = _truncate(descr, 120)

        text = (
            f"Relação {tipo_rel} entre {de} e {para} envolvendo a entidade {label}. "
            f"{descr_short}"
        )

        chunks.append({
            "entity": label,
            "filename": filename,
            "kind": "rel",
            "tipo_rel": tipo_rel,
            "de": de,
            "para": para,
            "descr": descr_short,
            "text": text,
        })

    # 4) CASO ESPECIAL: TIPOS AtividadeExec (1..16)
    if label == "AtividadeExec":
        tipos = no.get("tipos") or {}
        if isinstance(tipos, dict) and tipos:
            campos_por_tipo = {}
            for tipo_cod, info in tipos.items():
                props_tipo = (info.get("propriedades") or {})
                campos_por_tipo[str(tipo_cod)] = set(props_tipo.keys())

            all_campos = set().union(*campos_por_tipo.values())
            campos_comuns = set(all_campos)
            for s in campos_por_tipo.values():
                campos_comuns &= s
            campos_comuns = sorted(campos_comuns)

            for tipo_cod, info in tipos.items():
                desc = (info.get("descricao") or "").strip()
                desc_short = _truncate(desc, 256)
                exclusivos = sorted(
                    list(campos_por_tipo[str(tipo_cod)] - set(campos_comuns))
                )

                texto_partes = [f"Tipo de atividade {tipo_cod} da entidade {label}."]
                if desc_short:
                    texto_partes.append(desc_short)
                if exclusivos:
                    texto_partes.append(
                        "Campos exclusivos deste tipo: " + ", ".join(exclusivos) + "."
                    )
                if campos_comuns:
                    texto_partes.append(
                        "Campos comuns a todos os tipos: " + ", ".join(campos_comuns) + "."
                    )
                text = " ".join(texto_partes)

                chunks.append({
                    "entity": label,
                    "filename": filename,
                    "kind": "tipo",
                    "tipo_codigo": str(tipo_cod),
                    "name": f"tipo {tipo_cod}",
                    "descr": desc_short,
                    "campos_exclusivos": exclusivos,
                    "campos_comuns": campos_comuns,
                    "text": text,
                })

    return chunks


def _embed_text(text):
    resp = ollama.embeddings(
        model=EMBED_MODEL_NAME,
        prompt=text,
    )
    return resp["embedding"]


def rebuild_schema_index():
    """
    Recria COMPLETAMENTE o índice vetorial a partir de schema/*.json
    e salva em schema_vetorizado/.
    """
    all_chunks = []

    for ent, fname in ENTITY_TO_FILE.items():
        schema = _load_schema_json(fname)
        if not schema:
            continue
        chunks = _schema_to_chunks_for_embedding(schema, fname)
        all_chunks.extend(chunks)

    if not all_chunks:
        print("[WARN] Nenhum chunk de schema encontrado para vetorização.")
        return

    embeddings = []
    for ch in all_chunks:
        emb = _embed_text(ch["text"])
        embeddings.append(emb)

    emb_matrix = np.array(embeddings, dtype="float32")

    np.save(EMB_PATH, emb_matrix)
    with CHUNKS_PATH.open("w", encoding="utf-8") as f:
        json.dump(all_chunks, f, ensure_ascii=False)

    print(f"[INFO] Índice de schema reconstruído com {emb_matrix.shape[0]} chunks.")
    print(f"[INFO] Arquivo de vetores: {EMB_PATH}")
    print(f"[INFO] Arquivo de metadados: {CHUNKS_PATH}")


rebuild_schema_index()


[INFO] Índice de schema reconstruído com 261 chunks.
[INFO] Arquivo de vetores: schema_vetorizado\schema_emb.npy
[INFO] Arquivo de metadados: schema_vetorizado\schema_chunks.json
